In [ ]:
import warnings
# Ingnore warnings
warnings.filterwarnings("ignore")

import numpy as np
import scipy as sp
import sklearn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas.util.testing as tm
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

import os
import random
import math

In [ ]:
# Load datasets
data_folder = "./DA"

df_test_X = pd.read_csv(os.path.join(data_folder, "Test1.csv"))
df_test_Y = pd.read_csv(os.path.join(data_folder,"Test2.csv"))
df_train_X = pd.read_csv(os.path.join(data_folder,"Train1.csv"))
df_train_Y = pd.read_csv(os.path.join(data_folder, "Train2.csv"))


In [ ]:
test_X = df_test_X[df_test_X.columns[1:]]
train_X = df_train_X[df_train_X.columns[1:]]

In [ ]:
x_test = test_X.to_numpy()
x_train = train_X.to_numpy()

In [ ]:
y_test = test_Y.to_numpy()
y_train = train_Y.to_numpy()

In [ ]:

# dates = []
# scores = []

# with open('file.csv') as csvDataFile:
#     csvReader = csv.reader(csvDataFile)
#     for row in csvReader:
#         dates.append(row[0])
#         scores.append(row[1])

# print(dates)
# print(scores)

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import regularizers
max_features = 3 # 27205
max_len = 27205# 800

In [ ]:
BatchSize= 64 
ParamsLambdaRegularization = 2.5
HyperparametersEmbeddingDimension = 10
ParamsConvolutionNumber = 2 
ParamsConvolutionShape = "conic"
ParamsDenseShape = "conic"
ParamsFilterNumber = 64
Type = "CNN"
ParamsDenseNumber = 1
ParamsUnitNumber = 64
ParamsDropRate = 0.4
LearningRate = 0.0001
ParamsKernelSize = 9
Denominator = 800/BatchSize
LambdaRegularization = ParamsLambdaRegularization/Denominator
embedding_dim = 10
PoolStride= 5

In [ ]:
import keras
from keras.models import Sequential
from keras import layers, activations
from keras.optimizers import RMSprop
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(layers.Embedding(max_features, embedding_dim, input_length=max_len , embeddings_initializer="LecunNormal"))

if (ParamsConvolutionNumber > 0):
    for i in range(1,ParamsConvolutionNumber+1):
        print('i {}'.format(i))
        if ParamsConvolutionShape == "conic":
            FilterNumberL = ParamsFilterNumber*2**(i-1)
        elif (ParamsConvolutionShape == "rectangular"):
            FilterNumberL = ParamsFilterNumber

        if Type == "CNN":
            model.add(layers.Conv1D(filters = FilterNumberL, kernel_size = ParamsKernelSize, kernel_regularizer = regularizers.l2(LambdaRegularization),activation='relu'))
            checkpoint_filepath = os.path.join(data_folder,'best_cnn_model_test_val.hdf5')

        elif Type == "SNN":
            model.add(layers.Conv1D(filters = FilterNumberL, kernel_size = ParamsKernelSize, kernel_initializer = "LecunNormal",activation='selu'))
            checkpoint_filepath = os.path.join(data_folder,'best_snn_model.hdf5')


        model.add(layers.MaxPooling1D(pool_size = PoolStride))

     

model.add(layers.Flatten())           

if (ParamsDenseNumber > 0):
    for j in range(1,ParamsDenseNumber+1):
        if ParamsDenseShape == "conic":
            UnitNumberL = math.floor(ParamsUnitNumber*2**(j-1))
        elif ParamsDenseShape == "rectangular":
            UnitNumberL = ParamsUnitNumber


        if (Type == "CNN"):
            model.add(Dropout(ParamsDropRate))
            model.add(layers.Dense(units = UnitNumberL, kernel_regularizer=regularizers.l2(LambdaRegularization),activation='relu'))
            
        elif Type == "SNN":
            model.add(Dropout(ParamsDropRate))
            model.add(layers.Dense(units = UnitNumberL, kernel_regularizer=regularizers.l2(LambdaRegularization),activation='selu'))
           


if (Type == "CNN"):
    model.add(Dropout(ParamsDropRate))

elif(Type == "SNN"):
    model.add(AlphaDropout(ParamsDropRate))

model.add(layers.Dense(1))



model.summary()


i 1
i 2
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 27205, 10)         30        
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 27197, 64)         5824      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 5439, 64)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 5431, 128)         73856     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1086, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 139008)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 139008)   

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
model_checkpoint_callback = [checkpoint]

In [ ]:
opt = keras.optimizers.Adam(learning_rate = 1e-4)
model.compile(optimizer = opt,
              loss='mse',
              metrics=['mean_absolute_error'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
history = model.fit(train_X, train_Y,
                    epochs=1000,
                    batch_size=64,
                    # validation_split=0.2,
                    validation_data=(x_test, y_test),
                    callbacks=model_checkpoint_callback  
                    )

Epoch 1/1000
13/13 [==============================] - 77s 5s/step - loss: 1837.5101 - mean_absolute_error: 33.8694 - val_loss: 901.2878 - val_mean_absolute_error: 25.6308

Epoch 00001: val_loss improved from inf to 901.28784, saving model to /content/drive/My Drive/Colab Notebooks/DACCO/best_cnn_model_test_val.hdf5
Epoch 2/1000
13/13 [==============================] - 58s 4s/step - loss: 928.0071 - mean_absolute_error: 25.7506 - val_loss: 770.2194 - val_mean_absolute_error: 23.5358

Epoch 00002: val_loss improved from 901.28784 to 770.21936, saving model to /content/drive/My Drive/Colab Notebooks/DACCO/best_cnn_model_test_val.hdf5
Epoch 3/1000
13/13 [==============================] - 59s 4s/step - loss: 798.2887 - mean_absolute_error: 23.5820 - val_loss: 634.7170 - val_mean_absolute_error: 20.6319

Epoch 00003: val_loss improved from 770.21936 to 634.71704, saving model to /content/drive/My Drive/Colab Notebooks/DACCO/best_cnn_model_test_val.hdf5
Epoch 4/1000
13/13 [===================

In [ ]:
opt = keras.optimizers.Adam(learning_rate = 1e-4)
model.compile(optimizer = opt,
              loss='mse',
              metrics=['mean_absolute_error'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
history = model.fit(train_X, train_Y,
                    epochs=1000,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=model_checkpoint_callback  
                    )

Epoch 1/1000
10/10 [==============================] - 79s 6s/step - loss: 1948.0028 - mean_absolute_error: 34.5687 - val_loss: 1149.3130 - val_mean_absolute_error: 28.4367

Epoch 00001: val_loss improved from inf to 1149.31299, saving model to /content/drive/My Drive/Colab Notebooks/DACCO/best_cnn_model.hdf5
Epoch 2/1000
10/10 [==============================] - 62s 6s/step - loss: 1233.0068 - mean_absolute_error: 29.8891 - val_loss: 944.4062 - val_mean_absolute_error: 25.3811

Epoch 00002: val_loss improved from 1149.31299 to 944.40625, saving model to /content/drive/My Drive/Colab Notebooks/DACCO/best_cnn_model.hdf5
Epoch 3/1000
10/10 [==============================] - 62s 6s/step - loss: 879.4665 - mean_absolute_error: 24.3914 - val_loss: 763.6973 - val_mean_absolute_error: 23.5669

Epoch 00003: val_loss improved from 944.40625 to 763.69733, saving model to /content/drive/My Drive/Colab Notebooks/DACCO/best_cnn_model.hdf5
Epoch 4/1000
10/10 [==============================] - 62s 6s/s

In [ ]:
opt = keras.optimizers.Adam(learning_rate = 1e-4)
model.compile(optimizer = opt,
              loss='mean_squared_error',
              metrics=['mae'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
history = model.fit(train_X, train_Y,
                    epochs=1000,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=model_checkpoint_callback  
                    )

Epoch 1/1000
10/10 [==============================] - 73s 6s/step - loss: 1734.7352 - mae: 32.8010 - val_loss: 1163.1273 - val_mae: 28.5813
Epoch 2/1000
10/10 [==============================] - 56s 6s/step - loss: 1197.5088 - mae: 29.4028 - val_loss: 891.9045 - val_mae: 24.8216
Epoch 3/1000
10/10 [==============================] - 56s 6s/step - loss: 920.1581 - mae: 24.7995 - val_loss: 748.5442 - val_mae: 22.9498
Epoch 4/1000
10/10 [==============================] - 56s 6s/step - loss: 761.5400 - mae: 22.9081 - val_loss: 676.3766 - val_mae: 21.5409
Epoch 5/1000
10/10 [==============================] - 55s 6s/step - loss: 643.2063 - mae: 20.2943 - val_loss: 620.0079 - val_mae: 20.1023
Epoch 6/1000
10/10 [==============================] - 56s 6s/step - loss: 673.5240 - mae: 20.8485 - val_loss: 529.1867 - val_mae: 18.4418
Epoch 7/1000
10/10 [==============================] - 56s 6s/step - loss: 566.3809 - mae: 18.8748 - val_loss: 468.8873 - val_mae: 17.4023
Epoch 8/1000
10/10 [===========

KeyboardInterrupt: ignored